In [1]:
import utils_rag as ur
from rich.markdown import Markdown as rich_Markdown
from langchain_huggingface import HuggingFaceEmbeddings
import uuid
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_chroma import Chroma

In [ ]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI


if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "..."



llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)

### Carga Documentos

In [ ]:
file_name_aux_1 = 'Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.pdf'
text_chunks_1, tables_html_chunks_1, images_b64_chunks_1 = ur.cargar_chunks_proc(file_name_aux_1,'doc_1')

text_chunks_summary_1, tables_html_chunks_summary_1, images_b64_chunks_summary_1 = ur.cargar_summary_proc(file_name_aux_1,'doc_1')


## Vector Store

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

In [ ]:
path_retriver = "retriever"
ur.create_path_if_not_exists(path_retriver)

In [ ]:
# Rutas de persistencia
persist_dir = path_retriver + "/chroma_db_v1"   # Directorio para Chroma
docstore_file = path_retriver + "/docstore_v1.pkl"         # Archivo para el docstore


# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag"
                     , embedding_function=embed_model
                     ,persist_directory=persist_dir,)


# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in text_chunks_1]
print(f'ID de textos:{doc_ids}')
print(f'Numero de textos:{len(doc_ids)}')

summary_texts=[Document(page_content=doc_s['summary'],
           metadata={id_key: doc_ids[i]
                     ,'page_number':str(doc_s['page_number'])
                     ,'file_name':doc_s['filename']
                     ,'file_type':'Text'}) for i, doc_s in enumerate(text_chunks_summary_1)]


retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, text_chunks_1)))

In [ ]:
# Add tables
doc_ids = [str(uuid.uuid4()) for _ in tables_html_chunks_1]
print(f'ID de tablas:{doc_ids}')
print(f'Numero de tablas:{len(doc_ids)}')

summary_tables=[Document(page_content=doc_s['summary'],
           metadata={id_key: doc_ids[i]
                     ,'page_number':str(doc_s['page_number'])
                     ,'file_name':doc_s['filename']
                     ,'file_type':'Table'}) for i, doc_s in enumerate(tables_html_chunks_summary_1)]


retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(doc_ids, tables_html_chunks_1)))

In [ ]:
# Add images
doc_ids = [str(uuid.uuid4()) for _ in images_b64_chunks_1]
print(f'ID de imagenes:{doc_ids}')
print(f'Numero de imagenes:{len(doc_ids)}')
summary_images=[Document(page_content=doc_s['summary'],
           metadata={id_key: doc_ids[i]
                     ,'page_number':str(doc_s['page_number'])
                     ,'file_name':doc_s['filename']
                     ,'file_type':'Imagen'}) for i, doc_s in enumerate(images_b64_chunks_summary_1)]


retriever.vectorstore.add_documents(summary_images)
retriever.docstore.mset(list(zip(doc_ids, images_b64_chunks_1)))

In [ ]:
ur.save_docstore(store, docstore_file)

In [ ]:
query="que es deekseek?"
#response = query_expansion_response(query)
# 1. Recuperar documentos
docs = retriever.invoke(query, top_k=5)
print(docs)

In [ ]:
docs[1]

## Cargar Datos

In [3]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

path_retriver = "retriever"

# Rutas de persistencia
persist_dir = path_retriver + "/chroma_db_v1"   # Directorio para Chroma
docstore_file = path_retriver + "/docstore_v1.pkl"         # Archivo para el docstore

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="multi_modal_rag"
                     , embedding_function=embed_model
                     ,persist_directory=persist_dir,)


# The storage layer for the parent documents
store = ur.load_docstore(docstore_file)
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

## como se ajustan los documentos en un espacio vectorial

In [ ]:
import umap
import numpy as np
from tqdm import tqdm

embeddings = retriever.vectorstore.get(include=['embeddings'])['embeddings']

In [ ]:
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = ur.project_embeddings(embeddings, umap_transform)

In [ ]:
query="como es la codificacion automatica de la lista de espera?"
ur.plot_umap_embeddings(umap_transform,projected_dataset_embeddings, retriever, query)

In [ ]:
def query_expansion_response(query,llm):
    query = query
    messages = [
        ("system", "Eres un útil asistente experto en diversos tipos de archivos de animales y inteligencia artificial."
        " Proporciona un ejemplo de respuesta a la pregunta, que podría encontrarse en un documento especifico, la respuesta que"
        "sea corta y concisa."),
        ("user", query),
    ]
    response = llm.invoke(messages)
    return response.content


In [ ]:

query="como es la codificacion automatica de la lista de espera?"
query_exp = query_expansion_response(query,llm)

ur.plot_umap_embeddings(umap_transform,projected_dataset_embeddings, retriever, query+' '+query_exp)

## Re-ranking

In [ ]:
query = "como es el modelo de reconocimiento de enfermedades?"

a,b,c,d,e=ur.retrieved_documents(query
                    , retriever
                    , n_results=5
                    , reranker=True
                    , model_name_reranker='BAAI/bge-reranker-v2-m3'
                    , path_data = "data/docs/"
                    , print_results=True)

In [ ]:
query = "como es el modelo de reconocimiento de enfermedades?"

ur.retrieved_documents(query
                    , retriever
                    , n_results=5
                    , reranker=False
                    , model_name_reranker='BAAI/bge-reranker-v2-m3'
                    , path_data = "data/docs/"
                    , print_results=True)

In [4]:
query = "como es el modelo de reconocimiento de enfermedades?"

a,b,c,d,e=ur.retrieved_documents(query
                    , retriever
                    , n_results=5
                    , reranker=False
                    , model_name_reranker='BAAI/bge-reranker-v2-m3'
                    , path_data = "data/docs/"
                    , print_results=False)

In [6]:
def generar_citas(a_list, d_list, e_list,use_reranker=False):
    resultado = []
    for a_eval,d_val, e_val in zip(a_list,d_list, e_list):
        cita = {}
        if use_reranker:
            if d_val == 'Imagen':
                # Buscar el primer valor de a_list que coincide con d_val
                contenido = a_eval[1]
            elif d_val == 'Table':
                contenido = a_eval[1]
            else:
                contenido = e_val.get('text', '')
        else:
            if d_val == 'Imagen':
                # Buscar el primer valor de a_list que coincide con d_val
                contenido = a_eval
            elif d_val == 'Table':
                contenido = a_eval
            else:
                contenido = e_val.get('text', '')


        page_number = e_val.get('page_number', '')
        filename = e_val.get('filename', '')


        if contenido is not None:
            
            cita['contenido'] = contenido
            cita['page_number'] = page_number
            cita['filename'] = filename
            cita['file_type'] = d_val
            resultado.append(cita)
    
    contexto = "============================== Ventana de Contexto=======================================\n\n"
    for i,cita in enumerate(resultado):
        contexto += f"----------------------------Inicio Contexto {i+1}--------------------------------\n\n"
        contexto += f"Página: {cita['page_number']}\n\n"
        contexto += f"Archivo: {cita['filename']}\n\n"
        contexto += f"Tipo: {cita['file_type']}\n\n"
        contexto += f"Contenido: {cita['contenido']}\n\n"
        
        contexto += f"----------------------------Fin Contexto {i+1}--------------------------------\n\n"
    contexto += "==============================Fin Ventana de Contexto =======================================\n\n"

    
    return resultado, contexto



resultado, contexto = generar_citas(a, d, e)



In [7]:
contexto

'============================== Ventana de Contexto=======================================\n\n----------------------------Inicio Contexto 1--------------------------------\n\nPágina: [30, 31, 32, 33]\n\nArchivo: Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.pdf\n\nTipo: Text\n\nContenido: 3.3. Modelamiento del problema\nLa figura 3.2 muestra un esquema del flujo de la propuesta del modelo de codificación automática, y a continuación se explica en detalle los dos módulos principales del modelo; el NER y el motor de búsqueda (ElasticSearch).\nFigura 3.2: Propuesta de codificación automática de enfermedades.\n22\n3.3.1. Primer modulo: Reconocimiento automático de enferme- dades\nComo se muestra en la parte izquierda de la figura 3.2, la entrada del sistema es la inter- consulta escrita por el médico en forma de texto libre. Estos textos se utilizan como entrada para el modelo de reconocimiento automático de enfermedades. En particular

In [8]:
from rich.markdown import Markdown as rich_Markdown
rich_Markdown(contexto)

============================== Ventana de Contexto=======================================                          

----------------------------Inicio Contexto 1--------------------------------                                      

Página: [30, 31, 32, 33]                                                                                           

Archivo: Implementacion-y-aplicaciones-de-un-sistema-de-codificacion-automatica-de-la-lista-de-espera-chilena.pdf  

Tipo: Text                                                                                                         

Contenido: 3.3. Modelamiento del problema La figura 3.2 muestra un esquema del flujo de la propuesta del modelo de 
codificación automática, y a continuación se explica en detalle los dos módulos principales del modelo; el NER y el
motor de búsqueda (ElasticSearch). Figura 3.2: Propuesta de codificación automática de enfermedades. 22 3.3.1.     
Primer modulo: Reconocimiento automático de enferme- dades Como se muestra en la parte izquierda de la figura 3.2, 
la entrada del sistema es la inter- consulta escrita por el médico en forma de texto libre. Estos textos se        
utilizan como entrada para el modelo de reconocimiento automático de enfermedades. En particular, este modelo NER  
está basado en el trabajo propuesto de Rojas [38], donde se introduce una arquitectura simple pero altamente       
efectiva para el reconocimiento de entidades médicas. Esta arquitec- tura consiste en una red neuronal profunda    
compuesta de tres módulos principales, poniendo énfasis en la creación de buenas representaciones numéricas para la
secuencia de palabras que entra al modelo. Como se muestra en la figura 3.3, la primera capa denominada “Stacked   
embedding la- yer”, se encarga de transformar los textos asociados a los diagnósticos a una representación         
vectorial para que los algoritmos puedan operar con ellos. Luego, en la capa de codificación, se utiliza una red   
neuronal recurrente para obtener largas dependencias entre palabras en la oración, obteniendo así un mejor contexto
con el fin de mejorar las representaciones de la capa anterior. Finalmente, la capa de clasificación se encarga de 
asignar la etiqueta más probable a cada palabra del diagnóstico, logrando identificar qué partes del texto         
corresponden a una enfermedad. Figura 3.3: Propuesta de modelo de reconocimiento de enfermedades. Ex- traído de    
[38]. 23 Segundo modulo: Codificación automática de enfermedades Como se mencionó, el resultado del primer modulo  
del sistema es una lista que contiene todas las menciones de enfermedades en la interconsulta. En este segundo     
módulo, le asig- naremos un código de la nomenclatura CIE-10 a cada una estas menciones, el cuál puede ser         
utilizado posteriormente para decisiones clínicas o gestión. La asignación del código CIE-10 se realiza utilizando 
un motor de búsqueda que encuentra similitudes entre la mención de la enfermedad y los códigos en la lista tabular 
de la CIE-10. Este sistema se basa en Apache Elasticsearch, un motor de búsqueda y análisis de código abierto      
mantenido por la Apache Foundation 2. Este proceso ayuda a automatizar la asig- nación de códigos y reduce el      
tiempo y el esfuerzo necesarios para asignar manualmente los códigos a cada mención de enfermedad. A diferencia de 
los algoritmos de comparación de distancia entre palabras como por ejem- plo en el trabajo de Rojas et al. [39]    
donde se utiliza el algoritmo de distancia de Levensh- tein, este motor de búsqueda cuenta con un índice que       
contiene a cada una las enfermedades CIE-10 representadas a través de una serie de oraciones sinónimas extraídas   
desde distintas fuentes de información, simulando de mejor forma el proceso que siguen los expertos clínicos para  
determinar el código de una enfermedad. Las fuentes de información utilizadas para la extracción de definiciones   
sinónimas de las enfermedades fueron las siguientes

In [ ]:
resultado[0]['filename']

In [ ]:
path_datos = "data/docs/"
ur.plot_multiple_pages(path_datos + resultado[0]['filename'], resultado[0]['page_number'])